# Importing data into pandas

There are tons of ways you can get data into a pandas dataframe. Here are a few of the more common ones.

First, let's import pandas `as` pd.

In [2]:
import pandas as pd

### From a CSV file

If your data file is delimited with something other than a comma, you'll need to specify that in the `sep` argument. For example, if you had a pipe-delimited file: `pd.read_csv('../data/my-delimited-file.txt', sep='|')`

Let's read in the MLB salary data.

In [3]:
csv_df = pd.read_csv('../data/mlb.csv')

In [4]:
csv_df.head()

NAME TEAM POS    SALARY  START_YEAR  END_YEAR  YEARS
0   Clayton Kershaw  LAD  SP  33000000        2014      2020      7
1      Zack Greinke  ARI  SP  31876966        2016      2021      6
2       David Price  BOS  SP  30000000        2016      2022      7
3    Miguel Cabrera  DET  1B  28000000        2014      2023     10
4  Justin Verlander  DET  SP  28000000        2013      2019      7

### From a CSV file on the Internet

Just pass in the URL. This example uses the official results of the fall 2016 election in Nebraska.

The values that get returned aren't live -- like, if the results changed, your data frame would not update with new values. It reads in the data once.

In [5]:
csvi_df = pd.read_csv('http://electionresults.sos.ne.gov/resultsCSV.aspx?text=All')

In [6]:
csvi_df.head()

RaceID RaceName PartyCode AreaType  \
8862  For United States Senator - 6  Year Term      REP        SW      NaN   
8862  For United States Senator - 6  Year Term      REP        SW      NaN   
8862  For United States Senator - 6  Year Term      REP        SW      NaN   
8862  For United States Senator - 6  Year Term      REP        SW      NaN   
8862  For United States Senator - 6  Year Term      REP        SW      NaN   

      AreaNum  OfficeSeqNo  BallotOrder          CandidateID  \
8862      102          NaN           81         Jack Heidel    
8862      102          NaN          193         Deb Fischer    
8862      102          NaN         1253      Todd F. Watson    
8862      102          NaN         1269  Jeffrey Lynn Stein    
8862      102          NaN         1437  Dennis Frank Macek    

            CandidateName  CurrentDateTime  VoteFor  CandidateVotes  \
8862  8/8/2018 8:31:48 PM                1     9413        0.055667   
8862  8/8/2018 8:31:48 PM                1   128157        0.757904   
8862  8/8/2018 8:31:48 PM                1    19661        0.116273   
8862  8/8/2018 8:31:48 PM                1     6380        0.037730   
8862  8/8/2018 8:31:48 PM                1     5483        0.032426   

     CandidatePercentage PrecinctsReporting  PartialPrecinctsReporting  
8862           1464/1464             0/1464                        NaN  
8862           1464/1464             0/1464                        NaN  
8862           1464/1464             0/1464                        NaN  
8862           1464/1464             0/1464                        NaN  
8862           1464/1464             0/1464                        NaN

### From an Excel file

To read an Excel file in pandas, use the [`read_excel()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html) method. If you hadn't installed the [`xlrd`](http://www.python-excel.org/) module, you'd need to do that, as well. (We've already done so here.)

You might also want to specify the `sheet_name` to select your worksheet of interest -- the default is "the first one."

Here, we're reading in a spreadsheet with data on murders in large cities.

In [8]:
xl_df = pd.read_excel('../data/homicides2014.xlsx', sheet_name='Murders')

In [9]:
xl_df.head()

City State  Population  Murders
0  New York City     NY     8473938      333
1    Los Angeles     CA     3906772      260
2        Chicago     IL     2724121      415
3        Houston     TX     2219933      242
4   Philadelphia     PA     1559062      248

### From a Python data collection

Maybe the work you're doing in pandas happens downstream of some other Python processing, so the data exists as a native Python data collection -- say, a list of dictionaries. You can turn this (and other Python data collections, like a list of lists) into a pandas dataframe, too.

👉 For more details on Python data collections, [see this notebook](Python%20data%20types%20and%20basic%20syntax.ipynb#Collections-of-data).

In [7]:
test_data = [
    {'name': 'Cody Winchester', 'job': 'Training director', 'location': 'Colorado Springs, CO'},
    {'name': 'Jacob Sanders', 'job': 'Developer', 'location': 'Pittsburgh, PA'},
    {'name': 'Guy Fieri', 'job': 'Gourmand', 'location': 'Flavortown'},
    {'name': 'Sarah Huckabee Sanders', 'job': 'Spokeswoman', 'location': 'Washington, D.C.'}
]

py_df = pd.DataFrame(test_data)

In [8]:
py_df.head()

job              location                    name
0  Training director  Colorado Springs, CO         Cody Winchester
1          Developer        Pittsburgh, PA           Jacob Sanders
2           Gourmand            Flavortown               Guy Fieri
3        Spokeswoman      Washington, D.C.  Sarah Huckabee Sanders

### From an HTML table

OK SO.

This one requires you to install and specify the Python package that has the HTML parsing engine of your choice -- [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) or [lxml](http://lxml.de/). The default is `lxml`, but here we're going to use BeautifulSoup.

Huge caveat! Pulling data directly from an HTML table can be hit and miss, depending on how hairy the underlying HTML is. And if you want to scrape data from a website, it's usually better practice to save the results to a local file, _then_ load it up for analysis. But it's good to know that it's an option.

In this example, we've installed `BeautifulSoup` (alias `bs4`) with pipenv and we're going to import [a table of media witnesses](https://www.tdcj.state.tx.us/death_row/dr_media_witness_list.html) to Texas death row executions.

We're going to pass four things to [the pandas `read_html()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html):
1. The URL we want to scrape (in quotes, as a string)
2. The `flavor` of parser that we'd like to use to process the HTML (`bs4`)
3. The HTML attributes of the table we're targeting (in this case, the table has a `class` called `tdcj_table`)
4. The number of the list, in the list of lists that gets returned in a dataframe, that has the `header`? (Usually it's 0 -- the first one)

Reading through the documentation for this method, we also notice that this method returns a _list_ of matching tables as dataframes, so we need to grab the _first_ item in this list of tables returned. Our arguments were specific enough that there's only one item in the returned list, though, so we can just grab the first item with `[0]`.

👉For more details on selecting items from Python lists, see [this notebook](Python%20data%20types%20and%20basic%20syntax.ipynb#Collections-of-data).

In [9]:
html_df = pd.read_html('https://www.tdcj.state.tx.us/death_row/dr_media_witness_list.html',
                       flavor='bs4',
                       attrs={'class': 'tdcj_table'},
                       header=0)[0]

In [10]:
html_df.head()

Execution                  Link       Last Name   First Name TDCJ Number  \
0        553  Offender Information           Young  Christopher      999508   
1        552  Offender Information           Bible        Danny      999455   
2        551  Offender Information        Castillo         Juan      999502   
3        550  Offender Information          Davila        Erick      999545   
4        549  Offender Information  Rodriguez, III      Rosendo      999534   

         Date                                 Media Witness List  
0   7/17/2018  Michael Graczyk, Associated Press; Cody Stark,...  
1   6/27/2018  Michael Graczyk, Associated Press; Cody Stark,...  
2   4/16/2018  Michael Graczyk, Associated Press; Cody Stark,...  
3  04/25/2018  Michael Graczyk, Associated Press; Cody Stark,...  
4   3/27/2018  Michael Graczyk, Associated Press; Cody Stark,...

### From JSON

JSON stands for JavaScript Object Notation. It's a common data interchange format on the web. The [`read_json()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_json.html) method can pull JSON into a data frame.

Pandas can slurp in data from a local `.json` file, or from a URL -- say, a JSON API with data on dogs and cats registered in the Sunshine Coast Region of Australia. That one sounds fun -- let's do that.

In [11]:
json_df = pd.read_json('https://data.sunshinecoast.qld.gov.au/resource/44qj-t4fr.json')

In [12]:
json_df.head()

age  animaltype de_sexed gender               locality  \
0    0  D                 Y      M        LITTLE MOUNTAIN   
1    5  D                 Y      F         PELICAN WATERS   
2   12  D                 N      F              GHEERULLA   
3    4  D                 Y      M  GLASS HOUSE MOUNTAINS   
4    8  D                 Y      F       MOOLOOLAH VALLEY   

                  name primarybreed primarycolour specificbreed  
0                Boris       STAFFR    White         STAFFRDBT   
1  Molly (was Pandora)       MALTES    WhiteApric    MALTESEX    
2                  Mac       FOXTER    BlackTan      FOXTERRIEX  
3                  Jax       BOXER     RedWhite      BOXER       
4              Dililah       STAFFR    Brown         STAFFRDAM